# Четвертая неделя


#### Выполнил Ким Антон 
#### В рамках курса "Введение в машинное обучение" от Высшей школы экономики

## I. Линейная регрессия: прогноз оклада по описанию вакансии

### Введение
Линейные методы хорошо подходят для работы с разреженными данными — к таковым относятся, например, тексты. Это можно объяснить высокой скоростью обучения и небольшим количеством параметров, благодаря чему удается избежать переобучения.

Линейная регрессия имеет несколько разновидностей в зависимости от того, какой регуляризатор используется. Мы будем работать с гребневой регрессией, где применяется квадратичный, или L2-регуляризатор.

### План работы
<ul>
 <li>использовать линейную регрессию</li>
 <li>применять линейную регрессию к текстовым данным</li>
</ul>

### Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

### Загрузка данных 

In [2]:
data_train = pd.read_csv('data/week_4_salary-train.csv')
data_test = pd.read_csv('data/week_4_salary-test-mini.csv')

In [3]:
data_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [4]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   FullDescription     60000 non-null  object
 1   LocationNormalized  60000 non-null  object
 2   ContractTime        44418 non-null  object
 3   SalaryNormalized    60000 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.1+ MB


### Инструкции по выполнению
Для извлечения **TF-IDF**-признаков из текстов воспользуйтесь классом `sklearn.feature_extraction.text.TfidfVectorizer`.

Для предсказания целевой переменной мы будем использовать гребневую регрессию, которая реализована в классе `sklearn.linear_model.Ridge`.

Обратите внимание, что признаки **LocationNormalized** и **ContractTime** являются строковыми, и поэтому с ними нельзя работать напрямую. Такие нечисловые признаки с неупорядоченными значениями называют категориальными или номинальными. Типичный подход к их обработке — кодирование категориального признака с m возможными значениями с помощью m бинарных признаков. Каждый бинарный признак соответствует одному из возможных значений категориального признака и является индикатором того, что на данном объекте он принимает данное значение. Данный подход иногда называют **one-hot-кодированием**. Воспользуйтесь им, чтобы перекодировать признаки **LocationNormalized** и **ContractTime**. Он уже реализован в классе `sklearn.feature_extraction.DictVectorizer`. 

#### 1 Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv. Проведите предобработку:
<ul>
    <li>Приведите тексты к нижнему регистру.</li>
    <li>Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова.</li>
    <li>Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах</li>    
    <li>Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'.</li>
    <li>Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.</li>
    <li>Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными.</li>
</ul>

In [5]:
data_train['ContractTime'].fillna('nan', inplace=True)
data_train['FullDescription'] = data_train['FullDescription'].apply(lambda s: re.sub('[^a-z0-9]', 
                                                                                 ' ', s.lower()))
tfid = TfidfVectorizer(min_df=5)
X_train = tfid.fit_transform(data_train['FullDescription'])
X_test = tfid.transform(data_test['FullDescription'])

enc = DictVectorizer()
X_train_cat = enc.fit_transform(data_train[['ContractTime', 'LocationNormalized']].to_dict('records'))
X_test_cat = enc.transform(data_test[['ContractTime', 'LocationNormalized']].to_dict('records'))

In [6]:
train = hstack([X_train, 
               X_train_cat])
test = hstack([X_test, 
               X_test_cat])

#### 2. Обучите гребневую регрессию 
с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

In [7]:
model = Ridge()

model.fit(train, data_train['SalaryNormalized'])

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

#### 3. Постройте прогнозы для двух примеров из файла salary-test-mini.csv.

In [8]:
model.predict(test)

array([56582.8905928 , 37133.96512551])

## II. Составление фондового индекса

### Введение
Метод главных компонент (principal component analysis, PCA) — это один из методов обучения без учителя, который позволяет сформировать новые признаки, являющиеся линейными комбинациями старых. При этом новые признаки строятся так, чтобы сохранить как можно больше дисперсии в данных. Иными словами, метод главных компонент понижает размерность данных оптимальным с точки зрения сохранения дисперсии способом.

Основным параметром метода главных компонент является количество новых признаков. Как и в большинстве методов машинного обучения, нет четких рекомендаций по поводу выбора значения этого параметров. Один из подходов — выбирать минимальное число компонент, при котором объясняется не менее определенной доли дисперсии (это означает, что в выборке сохраняется данная доля от исходной дисперсии).

В этом задании понадобится измерять схожесть двух наборов величин. Если имеется набор пар измерений (например, одна пара — предсказания двух классификаторов для одного и того же объекта), то охарактеризовать их зависимость друг от друга можно с помощью корреляции Пирсона. Она принимает значения от -1 до 1 и показывает, насколько данные величины линейно зависимы. Если корреляция равна -1 или 1, то величины линейно выражаются друг через друга. Если она равна нулю, то линейная зависимость между величинами отсутствует.

### План работы
<ul>
    <li>работать с методом главных компонент</li>
    <li>использовать его для вычисления улучшенного индекса Доу-Джонса</li>
</ul>

### Подключение библиотек

In [9]:
from sklearn.decomposition import PCA

### Загрузка данных 

In [10]:
data = pd.read_csv('data/week_4_close_prices.csv')

### Описание данных
В этом задании мы будем работать с данными о стоимостях акций 30 крупнейших компаний США. На основе этих данных можно оценить состояние экономики, например, с помощью индекса Доу-Джонса. Со временем состав компаний, по которым строится индекс, меняется. Для набора данных был взят период с 23.09.2013 по 18.03.2015, в котором набор компаний был фиксирован (подробнее почитать о составе можно по ссылке из материалов).

Одним из существенных недостатков индекса Доу-Джонса является способ его вычисления — при подсчёте индекса цены входящих в него акций складываются, а потом делятся на поправочный коэффициент. В результате, даже если одна компания заметно меньше по капитализации, чем другая, но стоимость одной её акции выше, то она сильнее влияет на индекс. Даже большое процентное изменение цены относительно дешёвой акции может быть нивелировано незначительным в процентном отношении изменением цены более дорогой акции.


### Инструкции по выполнению
Метод главных компонент реализован в пакете `scikit-learn` в модуле `decomposition` в классе `PCA`. Основным параметром является количество компонент **(n_components)**. Для обученного преобразования этот класс позволяет вычислять различные характеристики. Например, поле **explained_variance_ratio_** содержит процент дисперсии, который объясняет каждая компонента. Поле **components_** содержит информацию о том, какой вклад вносят признаки в компоненты. Чтобы применить обученное преобразование к данным, можно воспользоваться методом **transform**.

Для нахождения коэффициента корреляции Пирсона можно воспользоваться функцией **corrcoef** из пакета numpy.

#### 1. На загруженных данных обучите преобразование PCA с числом компоненты равным 10. Скольких компонент хватит, чтобы объяснить 90% дисперсии?


In [11]:
data['date'] = data['date'].apply(lambda d: pd.to_datetime(d))

X = data.drop('date', axis=1)

In [12]:
model = PCA(n_components=10)

model.fit(X)

model.explained_variance_ratio_.shape[0]

10

#### 2. Примените построенное преобразование к исходным данным и возьмите значения первой компоненты.


In [13]:
test = model.transform(X)

#### 3. Загрузите информацию об индексе Доу-Джонса из файла djia_index.csv. Чему равна корреляция Пирсона между первой компонентой и индексом Доу-Джонса?

In [14]:
djia = pd.read_csv('data/week_4_djia_index.csv')

np.corrcoef(test[:,0], djia['^DJI'])[1][0]

0.9096522193050236

#### 4. Какая компания имеет наибольший вес в первой компоненте? Укажите ее название с большой буквы.

In [15]:
pd.Series(dict(zip(X.columns,model.components_[0]))).sort_values(ascending=False)

V       0.579684
MMM     0.329616
UNH     0.321564
HD      0.288996
GS      0.251227
DIS     0.233906
NKE     0.211889
TRV     0.189480
BA      0.120645
DD      0.114090
INTC    0.093132
JNJ     0.091395
WMT     0.087161
PG      0.077732
MSFT    0.076230
MRK     0.071390
UTX     0.053683
CSCO    0.050484
JPM     0.046988
KO      0.029055
PFE     0.023092
AXP     0.016138
VZ      0.000109
GE     -0.006205
T      -0.007206
MCD    -0.026107
XOM    -0.042942
CAT    -0.051661
CVX    -0.125860
IBM    -0.264999
dtype: float64